### Egypt Houses Price. 

in this dataset we will predict the price of property in egypt using regression models , we have a good database, our database has 27361 values and 12 columns, here we will use the "Egypt_Houses_Price.csv" database that i get it using scraping , below you will see an analysis of the data, the processing of that data, and the use of machine learning models to achieve our goal.

knowing that Unknown in this dataset mean is NAN 

In [ ]:
#Main libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize': [15, 9]}, font_scale=1.3)

#Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#Algorithms
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

#Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# to suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1- Import The Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/egypt-houses-prices/Egypt_Houses_Price.csv')

In [ ]:
df.head()

In [ ]:
df.info()
#from the info we can see that we have problem with data type and we need to work on it
#we have some Null to deal whith

### 2- Dealing With Null And Duplicated Raws

In [ ]:
df['Area'] = df['Area'].replace('Unknown', np.nan)
df['Bedrooms'] = df['Bedrooms'].replace('Unknown', np.nan)
df['Bathrooms'] = df['Bathrooms'].replace('Unknown', np.nan)
df['Price'] = df['Price'].replace('Unknown', np.nan)
#changing Unkhowing data to NAN 

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
#we can delete the Null bc its not too much

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

# Processing Data

### 3- Fixing the Dtype for the columns

In [ ]:
df.info()

In [ ]:
df['Bedrooms'] = df['Bedrooms'].replace('10+',11)
df['Bathrooms'] = df['Bathrooms'].replace('10+',11)

In [ ]:
df['Bedrooms'] = df['Bedrooms'].astype(float).astype(int)
df['Bathrooms'] = df['Bathrooms'].astype(float).astype(int)
df['Area'] = df['Area'].astype(float).astype(int)
df['Price'] = df['Price'].astype(float).astype(int)

In [ ]:
print(df['Type'].unique())
print(df['Level'].unique())

### from serching info we get :
- that ( Duplex , Apartment , Studio ) type can be in different level 
- that ( Twin house , Town House , Stand Alone Villa ,Chalet ) type only on Ground level
- that ( Penthouse ) type is on Highest only

### because the Type of property we will change the level from int to str 
- Ground = 0
- 10+ = 11
- Highest = 12

In [ ]:
# we can delete all unknowing level data that have ( Duplex , Apartment , Studio ) property type 
df=df.drop(df[(df['Level']=='Unknown')&(df['Type']=='Duplex')].index)
df=df.drop(df[(df['Level']=='Unknown')&(df['Type']=='Apartment')].index)
df=df.drop(df[(df['Level']=='Unknown')&(df['Type']=='Studio')].index)

In [ ]:
df.loc[(df['Level']=='10+'),'Level'] = 11
df.loc[(df['Level']=='Highest'),'Level'] = 12
df.loc[(df['Level']=='Ground'),'Level'] = 0

In [ ]:
df.loc[(df['Type']=='Standalone Villa')|
       (df['Type']=='Town House')|
       (df['Type']=='Twin house')|(df['Type']=='Stand Alone Villa')|
       (df['Type']=='Chalet')|
       (df['Type']=='Twin House'),'Level'] = 0
df.loc[(df['Type']=='Penthouse'),'Level'] = 12

In [ ]:
df['Level'] = df['Level'].astype(float).astype(int)
df['Level']

# * Furnished Column
#### From delivery date we can Guess if the property is Furnished or not

In [ ]:
df['Delivery_Date'].unique()

In [ ]:
df[(df['Furnished']=='Unknown')&(df['Delivery_Date']!='Ready to move')&(df['Delivery_Date']!='Unknown')]
#we can replace the Furnished data here naturally with NO bc the Delicery date us to far away

In [ ]:
df.loc[(df['Furnished']=='Unknown')&(df['Delivery_Date']!='Ready to move')&(df['Delivery_Date']!='Unknown'),'Furnished'] = 'No'

In [ ]:
df['Delivery_Term'].unique()

In [ ]:
df.loc[(df['Furnished']=='Unknown')&(df['Delivery_Term']!='Finished')&(df['Delivery_Term']!='Unknown ')]
#we can replace the Furnished data here naturally with NO bc the Delicery Term is not finished yet

In [ ]:
df.loc[(df['Furnished']=='Unknown')&(df['Delivery_Term']!='Finished')&(df['Delivery_Term']!='Unknown '),'Furnished'] = 'No'

### Drop columns that have more than 30% of NANs

In [ ]:
furnished = len(df[df['Furnished'] == 'Unknown']) / len(df)
level = len(df[df['Level'] == 'Unknown']) / len(df)
compound = len(df[df['Compound'] == 'Unknown']) / len(df)
Payment_Option = len(df[df['Payment_Option'] == 'Unknown']) / len(df)
Delivery_Date = len(df[df['Delivery_Date'] == 'Unknown']) / len(df)
Delivery_Term = len(df[df['Delivery_Term'] == 'Unknown ']) / len(df)
City = len(df[df['City'] == 'Unknown']) / len(df)

In [ ]:
print('Furnished: ', furnished)
print('Level: ',level)
print('Compound: ',compound)
print('Payment_Option: ',Payment_Option)
print('Delivery_Date: ',Delivery_Date)
print('Delivery_Term: ',Delivery_Term)
print('City: ',City)

In [ ]:
df = df.drop('Compound',1)
df = df.drop('Delivery_Date',1)

In [ ]:
df.head()

In [ ]:
df['Furnished'] = df['Furnished'].replace('Unknown', np.nan)
df['Delivery_Term'] = df['Delivery_Term'].replace('Unknown ', np.nan)
df.dropna(inplace=True)

In [ ]:
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

# * Type Column

In [ ]:
df['Type'].unique()
#we need to unite the unique type of data

In [ ]:
df['Type'].value_counts()

In [ ]:
df.loc[(df['Type']=='Standalone Villa'),'Type'] = 'Stand Alone Villa'
df.loc[(df['Type']=='Twin house'),'Type'] = 'Twin House'

In [ ]:
df['Type'].value_counts()

# * City Column

#### Deleting Location_City that have less than 5 rows

- city with low data don't give enough help to the model it's better to drop it

In [ ]:
pd.set_option('display.max_rows',500)
df['City'].value_counts(ascending=True) 

In [ ]:
ind = df['City'].value_counts(dropna=False).keys().tolist()
val = df['City'].value_counts(dropna=False).tolist()
value_dict = list(zip(ind, val))

In [ ]:
lc_sm = []
y = 'Less'
for val,ind in value_dict:
    if ind <= 5:
        lc_sm.append(val)
    else :
        pass
def lcdlt(x):
    if x in lc_sm:
        return y
    else :
        return x

In [ ]:
df['City'] = df['City'].apply(lcdlt)

In [ ]:
df['City']

In [ ]:
df=df.drop(df[(df['City']=='Less')].index)

### price depend highly with the location so we will deduct outlines for every city manuly

In [ ]:
plt.subplots(figsize=(24, 128))
sns.boxplot(y='City', x='Price',data=df)

In [ ]:
lcc = df['City'].value_counts().keys().tolist()

In [ ]:
for x in lcc:
    Q1= df[(df['City']==x)]['Price'].quantile(0.25)
    Q3= df[(df['City']==x)]['Price'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.2 * IQR
    lower_bound = Q1 - 1.2 * IQR
    df=df.drop(df[(df['City']==x)&(df['Price']>=upper_bound)].index)
    df=df.drop(df[(df['City']==x)&(df['Price']<=lower_bound)].index)

In [ ]:
df['City'].unique()

In [ ]:
df=df.drop(df[df['City']=='(View phone number)'].index)

In [ ]:
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)

# * Area Column

In [ ]:
df['Area'].describe()

In [ ]:
#deleting some raws that doest make sensce like 4 rooms in 100 miters
df=df.drop(df[(df['Area']<=100)&(df['Bedrooms']>=4)].index)
df=df.drop(df[(df['Area']<=30)&(df['Type']!='Studio')].index)

In [ ]:
df[(df['Area']>=300)&(df['Price']<=2000000)&(df['Payment_Option']=='Cash')&(df['Delivery_Term']=='Finished')]
df=df.drop(df[(df['Area']>=300)&(df['Price']<=2000000)&(df['Payment_Option']=='Cash')&(df['Delivery_Term']=='Finished')].index)

# * Payment_Option Column


In [ ]:
def Price_range(x) :
    if x <= 1000000 : 
        return 'Low Price'
    elif x <= 3000000 :
        return 'Mid Price'
    else :
        return 'high Price'

In [ ]:
df['Price_range'] = df['Price'].apply(Price_range)

In [ ]:
df.groupby('Price_range')['Payment_Option'].value_counts()

# Visualization

In [ ]:
"""for col in df.columns:
    print(col,':',df[col].nunique())
    print(df[col].value_counts().nlargest(7))
    print('\n'+'*'*20+'\n')"""

In [ ]:
mp = df['City'].value_counts()[0:20].sort_values()
sns.barplot(y=mp.index,x=mp.values);
plt.title('The City With The Most Buildings ');

In [ ]:
lpm = df.groupby('City')['Price'].mean()[0:20].sort_values()
sns.barplot(y=lpm.index,x=lpm.values);
plt.title('The City With The Higher Buildings Price');

In [ ]:
lpp = df.groupby('Type')['Price'].mean().sort_values()
sns.barplot(y=lpp.index,x=lpp.values);
plt.title('Most Expensive Property Type Building');

In [ ]:
lpb = df.groupby('Type')['Bedrooms'].mean().sort_values()
sns.barplot(y=lpb.index,x=lpb.values);
plt.title('Property With Mean Bed Room Number');

In [ ]:
df['Furnished'].value_counts().plot(kind='pie');

In [ ]:
df.groupby('Furnished')['Price'].mean().sort_values().plot(kind='pie');

In [ ]:
ind=(df.groupby('City')['Price'].sum()/df.groupby('City')['Area'].sum()).sort_values(ascending=False)[0:30].index
vlu=(df.groupby('City')['Price'].sum()/df.groupby('City')['Area'].sum()).sort_values(ascending=False)[0:30].values
sns.barplot(data = df, x = ind ,y= vlu ,ci = None,order = ind);
plt.xticks(rotation=90);
plt.title('The City With The 1 Meter^2 Price');

# from Visualization we did we can say that most importal ponints:
- Price depend on the city highly then on space of the property.
- Furnished or Not Furnished not effect highly on Price.
- After Cairo Coastal cities the most expensive property. 

In [ ]:
sns.pairplot(df, vars = ['Price', 'Area'], height=5, aspect=1.3)

In [ ]:
sns.pairplot(df, vars = ['Price', 'Level'], height=5, aspect=1.3);

In [ ]:
df.hist(bins=50, figsize=(15, 15));

In [ ]:
sns.histplot(data=df,x='Price',bins=150);

In [ ]:
sns.boxplot(x='Area', data=df)
sns.stripplot(x='Area', data=df, color="#474646");

In [ ]:
sns.histplot(data = df, x ='Area', kde = True)

In [ ]:
#df.to_csv('New_houes_data.csv')

# Building The Models

In [ ]:
df = pd.get_dummies(df, columns = ['Type', 'Furnished','City' ,'Payment_Option','Delivery_Term'])
X = df.drop(columns = ['Price','Price_range'])
y = df[['Price']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size = 0.25,shuffle = True ,random_state = 404)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
def performance(model,X_train,y_train,y_pred,y_test):
    '''
    This function for calculating the performance of the model.
    INPUT:
    model: Regression Model. The regression model.
    x_train: numpy.ndarray. The training data in the x.
    y_train: pandas.core.series.Series. The training data in the y.
    y_pred: numpy.ndarray. The predicted data.
    y_test: pandas.core.series.Series. The actual data.
    OUTPUT:
    The model performance by different metrics.
    '''
    print('Training Score:',model.score(X_train,y_train))
    print('Testing Score:',r2_score(y_test,y_pred))
    print('Other Metrics In Testing Data: ')
    print('MSE:',mean_squared_error(y_test,y_pred))
    print('MAE:',mean_absolute_error(y_test,y_pred))

In [ ]:
#Fitting the model
lr = LinearRegression()
lr.fit(X_train,y_train)

#The predicted data
lr_pred = lr.predict(X_test)

In [ ]:
performance(lr,X_train,y_train,lr_pred,y_test)

In [ ]:
#Fitting the model
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)

#The predicted data
dt_pred = dt.predict(X_test)

#The performance
performance(dt,X_train,y_train,dt_pred,y_test)

In [ ]:
#Fitting the model
rf = RandomForestRegressor()
rf.fit(X_train,y_train.values.ravel())

#The predicted data
rf_pred = rf.predict(X_test)

#The performance
performance(rf,X_train,y_train,rf_pred,y_test)